In [1]:
import pandas as pd

# Komentarze commitów

In [2]:
commits = pd.read_pickle('../../data/01_data_from_db/commits.pkl')
commits.head()

,commit_id,committer_id,project_id,created_at
0,1,17,10,2012-04-08 02:29:18
1,2,18,10,2011-12-29 21:12:27
2,3,18,10,2011-12-29 21:11:10
3,4,17,10,2011-12-29 20:54:15
4,5,18,10,2011-12-29 20:29:49


In [3]:
commit_comments = pd.read_pickle('../../data/01_data_from_db/commit_comments.pkl')
commit_comments.head()

,commit_comment_id,commit_id,body,created_at
0,28479,53162,I posted this as a patch file several weeks ag...,2011-05-19 10:37:02
1,28480,53162,"In addition this fix should close issues 395,7...",2011-05-19 10:45:11
2,28481,53162,"Pitcrawler, take a look at those tickets that ...",2011-05-19 10:58:56
3,28482,53162,Well my fix was almost exactly the same (I use...,2011-05-19 12:49:00
4,28483,53162,"You made fix for deep wounds stacking, but i t...",2011-05-19 13:16:08


Połączenie z projektami na podstawie commitów

In [4]:
commit_comments_for_projects = pd.merge(commits, commit_comments, on=['commit_id'], sort=False) \
    .drop(columns={'commit_id', 'committer_id', 'created_at_x'}) \
    .rename(columns={'created_at_y': 'created_at'})
commit_comments_for_projects.head()

,project_id,commit_comment_id,body,created_at
0,8,63596,I think there is a missing file:\r\nhttps://gi...,2013-03-18 07:32:12
1,8,63600,Inscrutable huge changesets like this one tend...,2012-06-04 16:11:08
2,8,63601,Hi Dave -- this is the initial commit. Some of...,2012-06-04 16:42:17
3,8,63602,"Oh! The other commit was marked ""initial."" S...",2012-06-04 16:55:59
4,7,63526,It's a common convention to save internal glob...,2013-03-22 21:37:12


### Na ten moment nie wykonuję analizy tekstu - usuwam więc informacje o treści

In [5]:
commit_comments_for_projects['year'] = commit_comments_for_projects['created_at'].dt.year
commit_comments_for_projects['month'] = commit_comments_for_projects['created_at'].dt.month
commit_comments_for_projects.drop(columns={'body'}, inplace=True)
commit_comments_for_projects.head()

,project_id,commit_comment_id,created_at,year,month
0,8,63596,2013-03-18 07:32:12,2013,3
1,8,63600,2012-06-04 16:11:08,2012,6
2,8,63601,2012-06-04 16:42:17,2012,6
3,8,63602,2012-06-04 16:55:59,2012,6
4,7,63526,2013-03-22 21:37:12,2013,3


## Nowe komentarze commitów w miesiącu

In [6]:
new_commit_comments = commit_comments_for_projects.groupby(['project_id', 'year', 'month']).count().reset_index() \
    .drop(columns={'commit_comment_id'}) \
    .rename(columns={'created_at': 'new_commit_comments'})
new_commit_comments.head()

,project_id,year,month,new_commit_comments
0,2,2011,7,4
1,2,2012,2,1
2,2,2012,8,12
3,2,2012,9,3
4,2,2012,12,4


## Łączna liczba nowych komentarzy od początku projektu

In [7]:
total_commit_comments_list = []
for index, row in new_commit_comments.iterrows():
    total_commit_comments_list.append(new_commit_comments[(((row['year'] > new_commit_comments['year']) |
        ((row['year'] == new_commit_comments['year']) & (row['month'] >= new_commit_comments['month']))) &
        (row['project_id'] == new_commit_comments['project_id']))]['new_commit_comments'].sum())

In [8]:
new_commit_comments['total_commit_comments'] = pd.Series(total_commit_comments_list)

Sprawdzenie dla jednego z projektów

In [9]:
new_commit_comments[new_commit_comments['project_id'] == 2]

,project_id,year,month,new_commit_comments,total_commit_comments
0,2,2011,7,4,4
1,2,2012,2,1,5
2,2,2012,8,12,17
3,2,2012,9,3,20
4,2,2012,12,4,24
5,2,2013,1,6,30
6,2,2013,3,7,37
7,2,2013,4,1,38
8,2,2013,6,1,39


In [10]:
new_commit_comments.to_pickle('../../data/02_prepared_data/new_commit_comments.pkl')

# Komentarze pull requestów

In [11]:
pull_requests = pd.read_pickle('../../data/01_data_from_db/pull_requests.pkl')
pull_requests.head()

,pull_request_id,project_id,merged
0,1,4,1
1,2,4,0
2,3,3,0
3,4,3,1
4,5,3,1


In [12]:
pull_request_comments = pd.read_pickle('../../data/01_data_from_db/pull_request_comments.pkl')
pull_request_comments.head()

,pull_request_id,comment_id,body,created_at
0,5,2717312,I'm a little confused by this combination. Cou...,2013-01-21 19:38:15
1,5,2722940,The idea here is that the settings specified i...,2013-01-22 08:38:46
2,5,2723020,...I just test-ran this code again and realize...,2013-01-22 08:49:02
3,5,2724044,"Ok, I am pretty new to R so this was an intere...",2013-01-22 10:46:03
4,14,4776654,You might want to check `eventExpr == 0` just ...,2013-06-19 16:32:14


Połączenie z projektami na podstawie pull requestów

In [13]:
pull_request_comments_with_projects = pd.merge(pull_requests, pull_request_comments, on=['pull_request_id'], sort=False) \
    .drop(columns={'pull_request_id', 'merged', 'comment_id'})
pull_request_comments_with_projects.head()

,project_id,body,created_at
0,3,I'm a little confused by this combination. Cou...,2013-01-21 19:38:15
1,3,The idea here is that the settings specified i...,2013-01-22 08:38:46
2,3,...I just test-ran this code again and realize...,2013-01-22 08:49:02
3,3,"Ok, I am pretty new to R so this was an intere...",2013-01-22 10:46:03
4,7,"Since I'm in here, `sapply` is a bit dangerous...",2013-09-18 17:11:25


### Na ten moment nie wykonuję analizy tekstu - usuwam więc informacje o treści

In [14]:
pull_request_comments_with_projects['year'] = pull_request_comments_with_projects['created_at'].dt.year
pull_request_comments_with_projects['month'] = pull_request_comments_with_projects['created_at'].dt.month
pull_request_comments_with_projects.drop(columns={'body'}, inplace=True)
pull_request_comments_with_projects.head()

,project_id,created_at,year,month
0,3,2013-01-21 19:38:15,2013,1
1,3,2013-01-22 08:38:46,2013,1
2,3,2013-01-22 08:49:02,2013,1
3,3,2013-01-22 10:46:03,2013,1
4,7,2013-09-18 17:11:25,2013,9


## Nowe komentarze pull requestów w miesiacu

In [15]:
new_pull_request_comments = pull_request_comments_with_projects.groupby(['project_id', 'year', 'month']).count().reset_index() \
    .rename(columns={'created_at': 'new_pull_request_comments'})
new_pull_request_comments.head()

,project_id,year,month,new_pull_request_comments
0,1,2011,4,5
1,1,2011,6,8
2,1,2011,10,134
3,1,2011,11,158
4,1,2011,12,688


## Łączna liczba nowych komentarzy od początku projektu

In [16]:
total_pull_request_comments_list = []
for index, row in new_pull_request_comments.iterrows():
    total_pull_request_comments_list.append(new_pull_request_comments[(((row['year'] > new_pull_request_comments['year']) |
        ((row['year'] == new_pull_request_comments['year']) & (row['month'] >= new_pull_request_comments['month']))) &
        (row['project_id'] == new_pull_request_comments['project_id']))]['new_pull_request_comments'].sum())

In [17]:
new_pull_request_comments['total_pull_request_comments'] = pd.Series(total_pull_request_comments_list)

Sprawdzenie dla jednego z projektów

In [18]:
new_pull_request_comments[new_pull_request_comments['project_id'] == 6]

,project_id,year,month,new_pull_request_comments,total_pull_request_comments
46,6,2012,3,14,14
47,6,2012,4,5,19
48,6,2012,6,5,24
49,6,2012,7,59,83
50,6,2012,10,1,84
51,6,2012,11,5,89
52,6,2013,1,10,99
53,6,2013,4,5,104
54,6,2013,5,1,105
55,6,2013,6,2,107


In [19]:
new_pull_request_comments.to_pickle('../../data/02_prepared_data/new_pull_request_comments.pkl')